In [3]:
%pip install torch

  Using cached torch-2.3.1-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached fsspec-2024.6.0-py3-none-any.whl.metadata (11 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
Using cached torch-2.3.1-cp312-cp312-win_amd64.whl (159.7 MB)
Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
Using cached filelock-3.14.0-py3-none-any.whl (12 kB)
Using cached fsspec-2024.6.0-py3-none-any.whl (176 kB)
Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\DT User2\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\networkx\\classes\\filters.py'
Consider using the `--user` option or check the permissions.



In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv(r'C:\Users\DT User2\Desktop\creditcard.csv')

print(data.head())

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [2]:
print(data.isnull().sum())

print(data['Class'].value_counts())


Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64
Class
0    284315
1       492
Name: count, dtype: int64


In [3]:
X = data.drop('Class', axis=1)
y = data['Class']

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y.to_numpy(), dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [4]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(30, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 30),
            nn.Sigmoid()  
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = Autoencoder()


In [5]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_data, _ in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_data)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_data.size(0)
    train_loss /= len(train_loader.dataset)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss:.4f}')


Epoch [1/20], Loss: 0.8691
Epoch [2/20], Loss: 0.8033
Epoch [3/20], Loss: 0.7934
Epoch [4/20], Loss: 0.7893
Epoch [5/20], Loss: 0.7871
Epoch [6/20], Loss: 0.7853
Epoch [7/20], Loss: 0.7841
Epoch [8/20], Loss: 0.7830
Epoch [9/20], Loss: 0.7821
Epoch [10/20], Loss: 0.7802
Epoch [11/20], Loss: 0.7792
Epoch [12/20], Loss: 0.7783
Epoch [13/20], Loss: 0.7771
Epoch [14/20], Loss: 0.7759
Epoch [15/20], Loss: 0.7730
Epoch [16/20], Loss: 0.7717
Epoch [17/20], Loss: 0.7709
Epoch [18/20], Loss: 0.7703
Epoch [19/20], Loss: 0.7699
Epoch [20/20], Loss: 0.7696


In [6]:
model.eval()
test_loss = 0
predictions = []
with torch.no_grad():
    for batch_data, _ in test_loader:
        reconstructions = model(batch_data)
        loss = criterion(reconstructions, batch_data)
        test_loss += loss.item() * batch_data.size(0)
        predictions.extend(reconstructions)

test_loss /= len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')

predictions = torch.stack(predictions).numpy()

reconstruction_errors = np.linalg.norm(X_test.numpy() - predictions, axis=1)

threshold = np.percentile(reconstruction_errors, 95)

anomalies = reconstruction_errors > threshold

print(f'Number of detected anomalies: {anomalies.sum()}')


Test Loss: 0.7565
Number of detected anomalies: 2849


In [7]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test.numpy(), anomalies))
print(classification_report(y_test.numpy(), anomalies))


[[54104  2760]
 [    9    89]]
              precision    recall  f1-score   support

         0.0       1.00      0.95      0.98     56864
         1.0       0.03      0.91      0.06        98

    accuracy                           0.95     56962
   macro avg       0.52      0.93      0.52     56962
weighted avg       1.00      0.95      0.97     56962

